# EDA

In [1]:
import numpy as np
import pandas as pd

import pyspark as ps

In [2]:
spark = (ps.sql.SparkSession.builder 
        .master("local[4]") 
        .appName("mov_rec") 
        .getOrCreate()
        )
sc = spark.sparkContext

In [3]:
rat_rdd = spark.read.format('com.databricks.spark.csv').\
                            options(header='true',\
                            inferschema='true').\
                            load('data/ml-latest-small/ratings.csv', header=True)
        

In [4]:
rat_rdd.show(5)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows



In [5]:
rat_rdd.tail(5)

[Row(userId=610, movieId=166534, rating=4.0, timestamp=1493848402),
 Row(userId=610, movieId=168248, rating=5.0, timestamp=1493850091),
 Row(userId=610, movieId=168250, rating=5.0, timestamp=1494273047),
 Row(userId=610, movieId=168252, rating=5.0, timestamp=1493846352),
 Row(userId=610, movieId=170875, rating=3.0, timestamp=1493846415)]

In [6]:
rat_rdd.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [7]:
mov_rdd = spark.read.format('com.databricks.spark.csv').\
                            options(header='true',\
                            inferschema='true').\
                            load('data/ml-latest-small/movies.csv', header=True)
        

In [8]:
# tags_rdd = spark.read.format('com.databricks.spark.csv').\
#                              options(header='true',\
#                              inferschema='true').\
#                              load('data/ml-latest-small/tags.csv', header=True)
        

In [9]:
mov_rdd.show(5)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows



In [10]:
rat_rdd.show(5)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows



In [11]:
mov_rdd.createOrReplaceTempView("mov_rdd")
rat_rdd.createOrReplaceTempView("rat_rdd")

In [12]:
# tags_rdd.show(5)

Leave out timestamp...

In [13]:
mov_surprise = spark.sql(
'''
SELECT ratings.userId, movies.title, ratings.rating 
  FROM mov_rdd AS movies
  JOIN rat_rdd AS ratings
    ON movies.movieId = ratings.movieId
'''
)

In [14]:
mov_surprise.createOrReplaceTempView("mov_surprise")

In [15]:
mov_surprise.show(5)

+------+--------------------+------+
|userId|               title|rating|
+------+--------------------+------+
|     1|    Toy Story (1995)|   4.0|
|     1|Grumpier Old Men ...|   4.0|
|     1|         Heat (1995)|   4.0|
|     1|Seven (a.k.a. Se7...|   5.0|
|     1|Usual Suspects, T...|   5.0|
+------+--------------------+------+
only showing top 5 rows



# Similarity Based Recommender using Surprise

Begin by reimplementing the similarity based recommender.

In [16]:
# # install surprise if not already available
# pip install surprise

In [17]:
from surprise import KNNBasic, KNNWithMeans, SVD

from surprise import Dataset
from surprise import Reader

from surprise.model_selection import cross_validate

# mov_surprise doesn't contain any zero values
mov_surprise = mov_surprise[mov_surprise['rating']!=0]

# ratings in dataset are from 0.5 to 5
reader = Reader(rating_scale=(0.5, 5))

In [18]:
mov_surprise.head(5)

[Row(userId=1, title='Toy Story (1995)', rating=4.0),
 Row(userId=1, title='Grumpier Old Men (1995)', rating=4.0),
 Row(userId=1, title='Heat (1995)', rating=4.0),
 Row(userId=1, title='Seven (a.k.a. Se7en) (1995)', rating=5.0),
 Row(userId=1, title='Usual Suspects, The (1995)', rating=5.0)]

In [19]:
# convert to pandas DataFrame
# warning, if file is too large, this will likely crash...
mov_pDF = mov_surprise.toPandas()
print(mov_pDF.head(5))

surprise_movie_ratings = Dataset.load_from_df(mov_pDF, reader)

   userId                        title  rating
0       1             Toy Story (1995)     4.0
1       1      Grumpier Old Men (1995)     4.0
2       1                  Heat (1995)     4.0
3       1  Seven (a.k.a. Se7en) (1995)     5.0
4       1   Usual Suspects, The (1995)     5.0


In [20]:
print(mov_pDF.tail(5))

        userId                           title  rating
100831     610                    Split (2017)     4.0
100832     610   John Wick: Chapter Two (2017)     5.0
100833     610                  Get Out (2017)     5.0
100834     610                    Logan (2017)     5.0
100835     610  The Fate of the Furious (2017)     3.0


In [21]:
# treat data as training set
training_movie_data = surprise_movie_ratings.build_full_trainset()

# anything not a user-item rating pair as test
# no data for this, so we can't do an eval on out of sample accuracy
testing_movie_data = training_movie_data.build_anti_testset()

# create instance of recommender algorithm object
# algo = KNNBasic()
algo = SVD()

# fit it on the training data
algo.fit(training_movie_data)

# predict on the test data
predictions = algo.test(testing_movie_data)

In [22]:
# print the first 5 predictions
predictions[:5]

[Prediction(uid=1, iid='Shawshank Redemption, The (1994)', r_ui=3.501556983616962, est=5, details={'was_impossible': False}),
 Prediction(uid=1, iid='Good Will Hunting (1997)', r_ui=3.501556983616962, est=4.970368517036985, details={'was_impossible': False}),
 Prediction(uid=1, iid='Kill Bill: Vol. 1 (2003)', r_ui=3.501556983616962, est=4.479092673886717, details={'was_impossible': False}),
 Prediction(uid=1, iid='Collateral (2004)', r_ui=3.501556983616962, est=4.376627285588289, details={'was_impossible': False}),
 Prediction(uid=1, iid='Talladega Nights: The Ballad of Ricky Bobby (2006)', r_ui=3.501556983616962, est=4.107670348915339, details={'was_impossible': False})]

In [23]:
# print last 5 predictions
predictions[-5:]

[Prediction(uid=610, iid='United States of Leland, The (2003)', r_ui=3.501556983616962, est=3.805219808349313, details={'was_impossible': False}),
 Prediction(uid=610, iid='A Home at the End of the World (2004)', r_ui=3.501556983616962, est=3.732074077381513, details={'was_impossible': False}),
 Prediction(uid=610, iid='League of Ordinary Gentlemen, A (2004)', r_ui=3.501556983616962, est=3.717807376944012, details={'was_impossible': False}),
 Prediction(uid=610, iid='I Think I Love My Wife (2007)', r_ui=3.501556983616962, est=3.799160105062458, details={'was_impossible': False}),
 Prediction(uid=610, iid='Man of the Year (1995)', r_ui=3.501556983616962, est=3.3785048694359996, details={'was_impossible': False})]

## Cross Validate: SVD

In [24]:
# standard format for cross validation
# algorithm used, ratings dataset, metrics, number of cross validations, verbose controls how much output
cross_validate(SVD(n_factors=5, reg_all=0.005), 
               surprise_movie_ratings, 
               measures=['RMSE', 'MAE'], 
               cv=5, 
               verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8736  0.8630  0.8685  0.8679  0.8710  0.8688  0.0036  
MAE (testset)     0.6690  0.6633  0.6656  0.6688  0.6705  0.6675  0.0026  
Fit time          2.02    2.26    2.07    2.29    2.55    2.24    0.19    
Test time         0.21    0.20    0.20    0.20    0.18    0.20    0.01    


{'test_rmse': array([0.87361955, 0.86295701, 0.86845563, 0.86794451, 0.87103893]),
 'test_mae': array([0.6690286 , 0.6633225 , 0.66559146, 0.66880799, 0.6705248 ]),
 'fit_time': (2.0196876525878906,
  2.2553467750549316,
  2.0663254261016846,
  2.2940425872802734,
  2.549058437347412),
 'test_time': (0.20783162117004395,
  0.19540190696716309,
  0.19676828384399414,
  0.19631648063659668,
  0.18456315994262695)}

## Cross Validtate: KNNBasic

In [25]:
cross_validate(KNNBasic(k=5), 
               surprise_movie_ratings, 
               measures=['RMSE', 'MAE'], 
               cv=5, 
               verbose=True)


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9568  0.9609  0.9616  0.9618  0.9646  0.9611  0.0025  
MAE (testset)     0.7298  0.7343  0.7326  0.7319  0.7333  0.7324  0.0015  
Fit time          0.11    0.13    0.09    0.12    0.12    0.12    0.01    
Test time         1.40    1.35    1.35    1.39    1.39    1.38    0.02    


{'test_rmse': array([0.95676515, 0.96092338, 0.96158519, 0.96183638, 0.96458438]),
 'test_mae': array([0.72975214, 0.73428546, 0.73260049, 0.73192949, 0.73326675]),
 'fit_time': (0.11104655265808105,
  0.1339273452758789,
  0.09459590911865234,
  0.12422513961791992,
  0.12299323081970215),
 'test_time': (1.4042072296142578,
  1.349153995513916,
  1.3520667552947998,
  1.386218547821045,
  1.390495777130127)}

## Cross Validate: KNNWithMeans

In [26]:
cross_validate(KNNWithMeans(k=5), 
               surprise_movie_ratings, 
               measures=['RMSE', 'MAE'], 
               cv=5, 
               verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9287  0.9255  0.9381  0.9281  0.9274  0.9295  0.0044  
MAE (testset)     0.7158  0.7115  0.7226  0.7126  0.7117  0.7148  0.0042  
Fit time          0.15    0.17    0.19    0.17    0.17    0.17    0.01    
Test time         1.45    1.50    1.45    1.46    1.52    1.48    0.03    


{'test_rmse': array([0.92867244, 0.92546161, 0.93808773, 0.92805408, 0.92739212]),
 'test_mae': array([0.71578055, 0.71152716, 0.72260049, 0.71260296, 0.71167512]),
 'fit_time': (0.14972925186157227,
  0.16548705101013184,
  0.19064569473266602,
  0.17007017135620117,
  0.16504764556884766),
 'test_time': (1.4516117572784424,
  1.5003068447113037,
  1.4511616230010986,
  1.4645960330963135,
  1.516674518585205)}

# Comparing to Baseline Model

In [27]:
from src import baselines 

# provide basline model our movie ratings
data = surprise_movie_ratings

print("\nGlobal Mean...")
algo_GM = baselines.GlobalMean()
cross_validate(algo_GM, data)

print("\nMeanOfMeans...")
algo_MM = baselines.MeanofMeans()
cross_validate(algo_MM, data)


Global Mean...

MeanOfMeans...


{'test_rmse': array([0.92830858, 0.93106667, 0.9341514 , 0.93381002, 0.92575874]),
 'test_mae': array([0.73156361, 0.73341146, 0.7320475 , 0.73516945, 0.72673657]),
 'fit_time': (0.6377971172332764,
  0.6517713069915771,
  0.6712062358856201,
  0.6668775081634521,
  0.6583433151245117),
 'test_time': (0.4629383087158203,
  0.4495258331298828,
  0.4611637592315674,
  0.4724736213684082,
  0.4600648880004883)}

# Predict results for user & movie

In [28]:
mov_pDF['title'].unique()

array(['Toy Story (1995)', 'Grumpier Old Men (1995)', 'Heat (1995)', ...,
       'Hazard (2005)', 'Blair Witch (2016)', '31 (2016)'], dtype=object)

## Search for Movie Title

In [29]:
def movie_title_search(str):
    
    title_lst = list()
    
    for i in mov_pDF['title'].unique():
        
        if str in i:
            title_lst.append(i)
    
    return title_lst  

print(movie_title_search('Matrix'))

['Matrix, The (1999)', 'Matrix Reloaded, The (2003)', 'Matrix Revolutions, The (2003)']


## Top N Recommendations for each user

Source:
- [Top N Recommendations](https://surprise.readthedocs.io/en/stable/FAQ.html#top-n-recommendations-py)  

In [32]:
from collections import defaultdict

def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

# # First train an SVD algorithm on the movielens dataset.
# data = Dataset.load_builtin('ml-100k')
# trainset = data.build_full_trainset()
# algo = SVD()
# algo.fit(trainset)

# Predict ratings for all pairs (u, i) that are NOT in the training set.
# predictions has been instantiated above already

top_n = get_top_n(predictions, n=10)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])
    print("\n")

1 ['Shawshank Redemption, The (1994)', 'Departed, The (2006)', 'Rear Window (1954)', 'Casablanca (1942)', 'Bonnie and Clyde (1967)', 'Amadeus (1984)', 'High Noon (1952)', 'Life Is Beautiful (La Vita è bella) (1997)', 'Sixth Sense, The (1999)', 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)']


2 ['Lawrence of Arabia (1962)', 'Streetcar Named Desire, A (1951)', 'Pulp Fiction (1994)', 'In the Name of the Father (1993)', 'Princess Bride, The (1987)', 'Paths of Glory (1957)', 'Casablanca (1942)', 'Cool Hand Luke (1967)', 'Amadeus (1984)', "One Flew Over the Cuckoo's Nest (1975)"]


3 ['Amadeus (1984)', 'Spotlight (2015)', 'Princess Bride, The (1987)', 'Shining, The (1980)', 'Legend of Drunken Master, The (Jui kuen II) (1994)', 'Notorious (1946)', 'Dead Poets Society (1989)', 'Rear Window (1954)', 'Harry Potter and the Prisoner of Azkaban (2004)', "One Flew Over the Cuckoo's Nest (1975)"]


4 ['Seventh Seal, The (Sjunde inseglet, Det) (1957)', 'Lord of the Ring